In [1]:
import pickle

import numpy as np
import open3d as o3d

import matplotlib.pyplot as plt
from matplotlib import cm

import ipywidgets as widgets
from IPython.display import display

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [20]:
from tag_mapping.evaluation import LatticeNavigationGraph
from tag_mapping.utils import box_to_linemesh, LineMesh

In [3]:
# for interactive dropdown selection
def on_dropdown_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        selected_option = change['new']

### Load the output of the evaluation

In [4]:
EVAL_OUTPUT_PATH = '/home/rsl_admin/openscene/comparison_outputs/test_eval_outputs/object/2t7WUuJeko7.pkl'

# EVAL_OUTPUT_PATH = '/home/rsl_admin/openscene/comparison_outputs/test_eval_outputs/region/2t7WUuJeko7.pkl'


In [5]:
with open(EVAL_OUTPUT_PATH, 'rb') as f:
    eval_output = pickle.load(f)

Load and visualize the mesh

In [6]:
scene_mesh = o3d.io.read_triangle_mesh(eval_output["ply_file_path"])

In [7]:
o3d.visualization.draw_geometries([scene_mesh])

Load lattice graph

In [8]:
lattice_graph = LatticeNavigationGraph.load(eval_output["lattice_graph_path"])

Load other outputs

In [9]:
label_gt_boxes = eval_output["label_gt_boxes"]
label_lattice_inds = eval_output["label_lattice_inds"]

label_proposals = eval_output["label_proposals"]

Evaluation output stores box corners instead of the axis-aligned bounding box object as it's not pickleable, create the boxes again here

In [10]:
for label, proposals in label_proposals.items():
    proposals["boxes"] = []
    for box_corners in proposals["boxes_corners"]:
        min_bound = np.min(box_corners, axis=0)
        max_bound = np.max(box_corners, axis=0)
        proposals["boxes"].append(
            o3d.geometry.AxisAlignedBoundingBox(min_bound, max_bound)
        )

### Visualize labels and proposals

In [11]:
select_mpcat40_label_dropdown = widgets.Dropdown(
    options=sorted(label_proposals.keys()),
    description='',
    disabled=False,
)
display(select_mpcat40_label_dropdown)

Dropdown(options=('bathtub', 'bed', 'cabinet', 'chair', 'chest_of_drawers', 'clothes', 'counter', 'cushion', '…

In [12]:
selected_label = select_mpcat40_label_dropdown.value
tag_options = list(set(label_proposals[selected_label]["tags"]))

select_tag_dropdown = widgets.Dropdown(
    options=['all'] + sorted(tag_options),
    description='',
    disabled=False,
)
display(select_tag_dropdown)

Dropdown(options=('all', 'n/a'), value='all')

In [15]:
label_boxes = label_gt_boxes[selected_label]

selected_tag = select_tag_dropdown.value
proposals = label_proposals[selected_label]


if len(proposals['boxes']) == 0:
    raise Exception("label has no associated proposals")


if selected_tag == 'all':
    selected_boxes = proposals["boxes"]
    selected_box_confidences = proposals["confidences"]
else:
    selected_boxes = [
        box for i, box in enumerate(proposals["boxes"]) if proposals["tags"][i] == selected_tag
    ]
    selected_box_confidences = [
        conf for i, conf in enumerate(proposals["confidences"]) if proposals["tags"][i] == selected_tag
    ]

max_conf = np.max(selected_box_confidences)

viz_selected_boxes_linemeshes = []
for box, conf in zip(selected_boxes, selected_box_confidences):
    color = cm.viridis(conf/max_conf)[:3]
    viz_selected_boxes_linemeshes += box_to_linemesh(
        box, 
        color=color, 
        radius=0.02
    ).cylinder_segments

    
label_boxes_linemeshes = []
for box in label_boxes:
    label_boxes_linemeshes += box_to_linemesh(
        box, 
        color=(0,1,0), 
        radius=0.02
    ).cylinder_segments
    

/home/rsl_admin/ram/map_anything/tag_mapping/tag_mapping/utils/line_mesh.py:21: RuntimeWarning: invalid value encountered in divide
  axis_ = axis_ / np.linalg.norm(axis_)


In [16]:
o3d.visualization.draw_geometries(
    [scene_mesh] + \
    label_boxes_linemeshes + \
    viz_selected_boxes_linemeshes
)

### Visualize the shortest paths for the proposals

In [17]:
select_mpcat40_label_dropdown = widgets.Dropdown(
    options=sorted(label_proposals.keys()),
    description='',
    disabled=False,
)
display(select_mpcat40_label_dropdown)

Dropdown(options=('bathtub', 'bed', 'cabinet', 'chair', 'chest_of_drawers', 'clothes', 'counter', 'cushion', '…

In [22]:
selected_label = select_mpcat40_label_dropdown.value
proposals = label_proposals[selected_label]

proposals_metrics = proposals['metrics']
proposals_confidences = proposals['confidences']

proposals_mean_spl = [metrics['mean_spl'] for metrics in proposals_metrics]

select_proposal_options = []
for i, (mean_spl, conf) in enumerate(zip(proposals_mean_spl, proposals_confidences)):
    option = f"{i} | mean spl: {mean_spl:.2f} | confidence: {conf}"
    select_proposal_options.append(option)

select_proposal_dropdown = widgets.Dropdown(
    options=select_proposal_options,
    description='',
    disabled=False,
)
display(select_proposal_dropdown)

Dropdown(options=('0 | mean spl: 11.00 | confidence: 1.0', '1 | mean spl: 9.00 | confidence: 1.0', '2 | mean s…

In [27]:
if len(proposals["boxes"]) == 0:
    raise Exception("label has no associated proposals")
    

selected_proposal_index = int(select_proposal_dropdown.value.split(' | ')[0])
print(f"selected proposal index: {selected_proposal_index}")

proposal_box = proposals["boxes"][selected_proposal_index]
proposal_box_linemesh = box_to_linemesh(
    proposal_box,
    color=(0,0,1),
    radius=0.02,
).cylinder_segments

proposal_node_inds = proposals["lattice_inds"][selected_proposal_index]
proposal_nodes_pcd = lattice_graph.o3d_nodes_pointcloud.select_by_index(proposal_node_inds)
proposal_nodes_pcd = proposal_nodes_pcd.paint_uniform_color((0,0,1))

label_boxes = label_gt_boxes[selected_label]
label_boxes_linemeshes = []
for box in label_boxes:
    label_boxes_linemeshes += box_to_linemesh(
        box, 
        color=(0,1,0), 
        radius=0.02
    ).cylinder_segments

label_node_inds = set()
for node_inds in label_lattice_inds[selected_label]:
        label_node_inds.update(node_inds)
label_node_inds = list(label_node_inds)
label_nodes_pcd = lattice_graph.o3d_nodes_pointcloud.select_by_index(label_node_inds)
label_nodes_pcd = label_nodes_pcd.paint_uniform_color((0,1,0))


shortest_path_linemeshes = []
for p_ind in proposal_node_inds:
    spl = np.inf
    matched_l_ind = None
    for l_ind in label_node_inds:
        
        new_spl = lattice_graph.shortest_path_length(p_ind, l_ind)
        if new_spl == None:
            continue
        
        if new_spl < spl:
            spl = new_spl
            matched_l_ind = l_ind
            
    if matched_l_ind != None:
        sp_inds = lattice_graph.shortest_path(p_ind, matched_l_ind)
        
        sp_lines = np.zeros((len(sp_inds)-1, 2)).astype(np.int32)
        sp_lines[:,0] = np.arange(len(sp_inds)-1)
        sp_lines[:,1] = 1 + np.arange(len(sp_inds)-1)
        
        sp_linemesh = LineMesh(
            points=lattice_graph.nodes_xyz[sp_inds],
            lines=sp_lines,
            colors=(0,1,1),
            radius=0.01,
        )
        
        shortest_path_linemeshes += sp_linemesh.cylinder_segments

        
o3d.visualization.draw_geometries(
    [scene_mesh] + \
    proposal_box_linemesh + \
    [proposal_nodes_pcd] + \
    label_boxes_linemeshes + \
    [label_nodes_pcd] + \
    shortest_path_linemeshes
)

selected proposal index: 5
[Open3D WARNING] The number of points is 0 when creating axis-aligned bounding box.
